# Testing HotVis on the flights paths

In [13]:
import csv
from HotVisFunctions import *
import pandas as pd
import numpy as np

In [14]:

def split_path_data(data: pp.PathData, training_percentile: float):
    training_data = pp.PathData(data.mapping)
    test_data = pp.PathData(data.mapping)
    split_indices = torch.rand(data.num_paths)
    for i in range(data.num_paths):
        if split_indices[i] <= training_percentile:
            training_data.append_walk(data.get_walk(i))
        else:
            test_data.append_walk(data.get_walk(i))
    return training_data, test_data


In [ ]:
def from_ngram(file: str, sep: str = ",") -> pp.PathData:
    with open(file, "r", encoding="utf-8") as f:
        paths = [line.strip().split(sep) for line in f if len(line.strip().split(sep)) > 1]

    mapping = pp.IndexMap()
    mapping.add_ids(np.concatenate([np.array(path) for path in paths]))

    pathdata = pp.PathData(mapping)
    pathdata.append_walks(node_seqs=paths, weights=[])

    return pathdata

In [ ]:
# load flights
flights = from_ngram("graphs/Flights/flights.ngram")
flights_train, flights_test = split_path_data(flights, 0.7)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7ffffc85e490>>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [8]:
print(flights_train.num_paths)
print(flights_test.num_paths)
print(flights_train.num_paths/flights.num_paths)
print(flights_test.num_paths/flights.num_paths)

200209
86601
0.6980544611415223
0.30194553885847775


In [9]:
layout_2 = HotVis(flights_train, 2, 50000, 3, alpha=[1, 0.5], force=10)
layout_3 =  HotVis(flights_train, 3, 50000, 3, alpha=[1, 0.5, 0.3], force=10)
layout_5 =  HotVis(flights_train, 5, 50000, 3, alpha=[1, 0.5, 0.3, 0.25, 0.2], force=10)

100%|██████████| 50000/50000 [00:33<00:00, 1494.35it/s]


In [10]:
print(edge_crossing(flights_test, layout_2))
print(edge_crossing(flights_test, layout_3))
print(edge_crossing(flights_test, layout_5))

50360
51062
50086


In [11]:
print(causal_path_dispersion(flights_test, layout_2, 3))
print(causal_path_dispersion(flights_test, layout_3, 3))
print(causal_path_dispersion(flights_test, layout_5, 3))

tensor(1.2095, dtype=torch.float64)
tensor(1.1756, dtype=torch.float64)
tensor(1.1736, dtype=torch.float64)


In [12]:
print(closeness_eccentricity(flights_test, layout_2, 3, 0.1))
print(closeness_eccentricity(flights_test, layout_3, 3, 0.1))
print(closeness_eccentricity(flights_test, layout_5, 3, 0.1))

KeyboardInterrupt: 